In [11]:
import pandas as pd
import numpy as np

In [12]:
sales = pd.read_csv("sales.csv", parse_dates=["date"])
sales

,date,city,product,qty,price
0,2020-01-01,Kraków,espresso,2,8.0
1,2020-01-01,Warszawa,latte,1,14.0
2,2020-01-02,Kraków,herbata,3,9.5
3,2020-01-02,Gdańsk,latte,2,13.0
4,2020-01-03,Kraków,espresso,1,8.5
5,2020-01-03,Warszawa,herbata,4,9.0
6,2020-01-04,Gdańsk,espresso,3,8.0
7,2020-01-04,Kraków,latte,2,NaN
8,2020-01-05,Warszawa,latte,3,13.5
9,2020-01-05,Gdańsk,herbata,1,10.0


In [13]:
#introspection
sales?

Type:        DataFrame
String form:
date      city   product  qty  price
           0  2020-01-01    Kraków  espresso    2    8.0
           1  20 <...>  10.0
           10 2020-01-06    Kraków  espresso    4    8.0
           11 2020-01-06  Warszawa   herbata    2    NaN
Length:      12
File:        ~/miniforge3/envs/ml/lib/python3.11/site-packages/pandas/core/frame.py
Docstring:  
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, dataclass or list-like objects. If
    data is a dict, column order follows insertion-order. If a dict contains Series
    which have an index defined, it is aligned by its index.

In [14]:
%%timeit
#computing total revenue as price * qty using a for loop
total = 0
for row in sales.itertuples():
    q = row.qty
    p = 0 if pd.isna(row.price) else row.price
    total += q * p

338 μs ± 10.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
%%timeit
#computing total revenue as price * qty using a vectorization
sum_vec = (sales["price"] * sales["qty"]).sum()

104 μs ± 9.13 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
#ndarray shape(n,2) from DataFrame
X = sales[["qty", "price"]].to_numpy()
X.shape

(12, 2)

In [17]:
#broadcastig a vector [1, 1.1] over X
X_increse = X * [1, 1.1]
X_increse

array([[ 2.  ,  8.8 ],
       [ 1.  , 15.4 ],
       [ 3.  , 10.45],
       [ 2.  , 14.3 ],
       [ 1.  ,  9.35],
       [ 4.  ,  9.9 ],
       [ 3.  ,  8.8 ],
       [ 2.  ,   nan],
       [ 3.  , 14.85],
       [ 1.  , 11.  ],
       [ 4.  ,  8.8 ],
       [ 2.  ,   nan]])

In [18]:
#computing new income as price * qty using a vectorization
potential_income = np.nansum(X_increse[:,0] * X_increse[:,1])
potential_income

np.float64(259.05)

In [19]:
#adding new column to sales
sales["revenue"] = sales.qty * sales.price
sales

,date,city,product,qty,price,revenue
0,2020-01-01,Kraków,espresso,2,8.0,16.0
1,2020-01-01,Warszawa,latte,1,14.0,14.0
2,2020-01-02,Kraków,herbata,3,9.5,28.5
3,2020-01-02,Gdańsk,latte,2,13.0,26.0
4,2020-01-03,Kraków,espresso,1,8.5,8.5
5,2020-01-03,Warszawa,herbata,4,9.0,36.0
6,2020-01-04,Gdańsk,espresso,3,8.0,24.0
7,2020-01-04,Kraków,latte,2,NaN,NaN
8,2020-01-05,Warszawa,latte,3,13.5,40.5
9,2020-01-05,Gdańsk,herbata,1,10.0,10.0


In [20]:
#filtr 2 or more espresso from Kraków 
filtered = sales[
    (sales['city'] == 'Kraków') &
    (sales['product'] == 'espresso') &
    (sales['qty'] >= 2)
]
filtered

,date,city,product,qty,price,revenue
0,2020-01-01,Kraków,espresso,2,8.0,16.0
10,2020-01-06,Kraków,espresso,4,8.0,32.0


In [21]:
#data sorted descending
filtered.revenue.sort_values(ascending=False)

10    32.0
0     16.0
Name: revenue, dtype: float64

In [22]:
#calculating the sum of poduct
total_product = sales["qty"].sum()
total_product

np.int64(28)

In [23]:
#calculating the average of revenue
arg_revenue = sales["revenue"].mean()
arg_revenue

np.float64(23.55)

In [24]:
#calculating the median of price
price_median = sales['price'].median()
price_median

np.float64(9.25)

In [25]:
#saving changing sales to csv
sales.to_csv("sales_clean.csv", index=False)

In [26]:
!cat sales_clean.csv

date,city,product,qty,price,revenue
2020-01-01,Kraków,espresso,2,8.0,16.0
2020-01-01,Warszawa,latte,1,14.0,14.0
2020-01-02,Kraków,herbata,3,9.5,28.5
2020-01-02,Gdańsk,latte,2,13.0,26.0
2020-01-03,Kraków,espresso,1,8.5,8.5
2020-01-03,Warszawa,herbata,4,9.0,36.0
2020-01-04,Gdańsk,espresso,3,8.0,24.0
2020-01-04,Kraków,latte,2,,
2020-01-05,Warszawa,latte,3,13.5,40.5
2020-01-05,Gdańsk,herbata,1,10.0,10.0
2020-01-06,Kraków,espresso,4,8.0,32.0
2020-01-06,Warszawa,herbata,2,,


In [27]:
#saving columns date, product, revenue of sales in json
sales_part = sales[["date", "product", "revenue"]]
sales_part.to_json("summary.json", orient="records")

In [28]:
!cat summary.json

[{"date":1577836800000,"product":"espresso","revenue":16.0},{"date":1577836800000,"product":"latte","revenue":14.0},{"date":1577923200000,"product":"herbata","revenue":28.5},{"date":1577923200000,"product":"latte","revenue":26.0},{"date":1578009600000,"product":"espresso","revenue":8.5},{"date":1578009600000,"product":"herbata","revenue":36.0},{"date":1578096000000,"product":"espresso","revenue":24.0},{"date":1578096000000,"product":"latte","revenue":null},{"date":1578182400000,"product":"latte","revenue":40.5},{"date":1578182400000,"product":"herbata","revenue":10.0},{"date":1578268800000,"product":"espresso","revenue":32.0},{"date":1578268800000,"product":"herbata","revenue":null}]

In [29]:
#reading csv and proving that column date is datetype type
sales_clean = pd.read_csv("sales_clean.csv", parse_dates=["date"])
sales_clean.dtypes

date       datetime64[ns]
city               object
product            object
qty                 int64
price             float64
revenue           float64
dtype: object